# Copy files

Brief 1-2 sentence description of notebook.

TODO: Supplement the description
- Notebook that extracts the timestamps and gets the time that tones played

In [1]:
# Imports of all used packages and libraries
import sys
import os
import glob
from collections import defaultdict

In [2]:
import zipfile


# Functions

## Inputs & Data

- Explanation of each input and where it comes from.

Inputs and Required data loading
- input variable names are in all caps snake case
- Whenever an input changes or is used for processing 
- The variables are all lower in snake case

In [3]:
# Path of the directory that contains the Spike Gadgets recording and the exported timestamp files
# Exported with this tool https://docs.spikegadgets.com/en/latest/basic/ExportFunctions.html
# Export these files:
    # -raw – Continuous raw band export.
    # -dio – Digital IO channel state change export.
    # -analogio – Continuous analog IO export.
# INPUT_DIR = "/scratch/back_up/reward_competition_extention/final_proc/phy_curation"
# INPUT_DIR = "/scratch/back_up/reward_competition_extention/data/rce_cohort_3"
INPUT_DIR = "/orange/ruogu.fang/data/UKB/brain"
OUTPUT_DIR = os.path.join("/blue/ruogu.fang/ryoi360/projects/fmri_vlm/data/UKB/brain/20227_rsfMRI_NIFTI") # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)


# Processing

# Copying the files over

In [4]:
import shutil
import glob
import os



In [5]:

# for p in sorted(glob.glob('*/*/*.npy', recursive=True, root_dir=INPUT_DIR)):
#     if "copies" in p:
#         continue
#     print(p)
#     if os.path.isfile(os.path.join(INPUT_DIR, p)):
#         os.makedirs(os.path.join(OUTPUT_DIR, os.path.dirname(p)), exist_ok=True)
#         if not os.path.exists(os.path.join(OUTPUT_DIR, p)):
#             shutil.copy(os.path.join(INPUT_DIR, p), os.path.join(OUTPUT_DIR, p))
#         else:
#             print("{} exists".format(os.path.join(OUTPUT_DIR, p)))



In [6]:
import pandas as pd

In [7]:
diagnosis_df = pd.read_pickle("./proc/trimmed_icd.pkl")

In [8]:
diagnosis_df

,eid,31-0.0,21022-0.0,all_diagnoses,mental_and_neural_diagnoses,has_any_diagnosis,has_mental_or_neural_diagnosis,has_PD,has_AD
0,1000014,1,63,"[I10, K20, K227, K30, K449, N300, R101, R31, R...",[],True,False,False,False
1,1000023,1,48,"[M179, N359, R31, Z302]",[],True,False,False,False
2,1000030,0,65,"[D051, D509, K224, K573, R69]",[],True,False,False,False
3,1000041,0,53,"[H269, H353, M2323]",[],True,False,False,False
4,1000059,0,48,[],[],False,False,False,False
...,...,...,...,...,...,...,...,...,...
502304,6025359,1,62,"[I251, M4806, N40, N47, R798]",[],True,False,False,False
502305,6025362,0,53,"[M169, Z038]",[],True,False,False,False
502306,6025377,1,58,[],[],False,False,False,False
502307,6025386,0,43,[],[],False,False,False,False


In [9]:
all_files = sorted(glob.glob('/orange/ruogu.fang/data/UKB/brain/20227_rsfMRI_NIFTI/rsfMRI_zip/*', recursive=True, root_dir=INPUT_DIR))

In [10]:
all_IDS = [os.path.basename(path).split("_")[0] for path in all_files]

In [11]:
all_IDS[:20]

['1000023',
 '1000086',
 '1000317',
 '1000503',
 '1000665',
 '1000768',
 '1000925',
 '1001109',
 '1001390',
 '1001631',
 '1001784',
 '1001851',
 '1001900',
 '1002112',
 '1002308',
 '1002819',
 '1003319',
 '1003639',
 '1003750',
 '1004075']

- Getting all the file paths for healthy ADs

In [12]:
# Convert diagnosis_df["eid"] to a set for O(1) lookups
eid_set = set(diagnosis_df["eid"].astype(str))

# Use list comprehension with set lookup
filtered_IDS = [string for string in all_IDS if string in eid_set]

In [13]:
len(filtered_IDS)

22281

In [14]:
len(all_files)

22288

# Copying files over

In [15]:
import zipfile
import os

In [16]:
INPUT_DIR

'/orange/ruogu.fang/data/UKB/brain'

In [17]:
OUTPUT_DIR

'/blue/ruogu.fang/ryoi360/projects/fmri_vlm/data/UKB/brain/20227_rsfMRI_NIFTI'

In [18]:
# Define the target file path inside each zip
TARGET_FILE = "fMRI/rfMRI.ica/filtered_func_data_clean.nii.gz"

for subject_ID in filtered_IDS:
    for zip_path in sorted(glob.glob(f"{INPUT_DIR}/20227_rsfMRI_NIFTI/**/*{subject_ID}*.zip", recursive=True)):
        print(f"Processing: {zip_path}")

        # Check if the zip file exists and is a valid zip
        if not os.path.isfile(zip_path):
            print(f"Error: ZIP file not found: {zip_path}")
            continue  # Skip this file and move to the next

        try:
            # Open the ZIP file
            with zipfile.ZipFile(zip_path, "r") as zip_ref:
                # Check if the target file exists in the ZIP archive
                if TARGET_FILE in zip_ref.namelist():
                    # Define the extraction path
                    subject_output_dir = os.path.join(OUTPUT_DIR, subject_ID)
                    os.makedirs(subject_output_dir, exist_ok=True)

                    # Extract only the target file
                    extracted_path = zip_ref.extract(TARGET_FILE, subject_output_dir)
                    print(f"Extracted: {extracted_path}")
                else:
                    print(f"Target file not found in {zip_path}")
        except zipfile.BadZipFile:
            print(f"Error: Invalid ZIP file: {zip_path}")
        except zipfile.LargeZipFile:
            print(f"Error: ZIP file is too large and requires ZIP64 support: {zip_path}")
        except Exception as e:
            print(f"Unexpected error processing {zip_path}: {str(e)}")


Processing: /orange/ruogu.fang/data/UKB/brain/20227_rsfMRI_NIFTI/rsfMRI_zip/1000023_20227_2_0.zip
Extracted: /blue/ruogu.fang/ryoi360/projects/fmri_vlm/data/UKB/brain/20227_rsfMRI_NIFTI/1000023/fMRI/rfMRI.ica/filtered_func_data_clean.nii.gz
Processing: /orange/ruogu.fang/data/UKB/brain/20227_rsfMRI_NIFTI/rsfMRI_zip/1000086_20227_2_0.zip
Extracted: /blue/ruogu.fang/ryoi360/projects/fmri_vlm/data/UKB/brain/20227_rsfMRI_NIFTI/1000086/fMRI/rfMRI.ica/filtered_func_data_clean.nii.gz
Processing: /orange/ruogu.fang/data/UKB/brain/20227_rsfMRI_NIFTI/rsfMRI_zip/1000317_20227_2_0.zip
Extracted: /blue/ruogu.fang/ryoi360/projects/fmri_vlm/data/UKB/brain/20227_rsfMRI_NIFTI/1000317/fMRI/rfMRI.ica/filtered_func_data_clean.nii.gz
Processing: /orange/ruogu.fang/data/UKB/brain/20227_rsfMRI_NIFTI/rsfMRI_zip/1000503_20227_2_0.zip
Extracted: /blue/ruogu.fang/ryoi360/projects/fmri_vlm/data/UKB/brain/20227_rsfMRI_NIFTI/1000503/fMRI/rfMRI.ica/filtered_func_data_clean.nii.gz
Processing: /orange/ruogu.fang/data/

KeyboardInterrupt: 

In [ ]:
def unzip_files_recursively(root_dir):
    """
    Recursively walk through root_dir, find .zip files, and unzip them.
    If a .zip file is in a folder containing 'zip' in its name, it will be
    extracted to a parallel folder with 'unzip' in its name instead.
    """
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.lower().endswith('.zip'):
                zip_file_path = os.path.join(dirpath, filename)
                
                # Remove the .zip extension to get the extraction folder name
                extraction_folder_name = os.path.splitext(filename)[0]
                
                # Check if the parent folder name contains 'zip'
                parent_folder = os.path.basename(dirpath)
                if 'zip' in parent_folder.lower():
                    # Replace 'zip' with 'unzip' in the parent folder name
                    new_parent_folder = parent_folder.lower().replace('zip', 'unzip')
                    
                    # The parent of the parent folder
                    parent_of_parent = os.path.dirname(dirpath)
                    
                    # Construct the path to the new folder
                    new_unzip_folder_path = os.path.join(parent_of_parent, new_parent_folder)
                    
                    # Final path: new folder + subfolder named after the ZIP file
                    final_extraction_path = os.path.join(new_unzip_folder_path, extraction_folder_name)
                    
                else:
                    # If 'zip' is not in the parent folder, just unzip in the same folder
                    final_extraction_path = os.path.join(dirpath, extraction_folder_name)
                
                # Make sure the target extraction directory exists
                os.makedirs(final_extraction_path, exist_ok=True)
                
                # Unzip the file
                with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                    zip_ref.extractall(final_extraction_path)
                
                print(f"Unzipped '{zip_file_path}' to '{final_extraction_path}'")

In [ ]:
# unzip_files_recursively("../../data/UKB/brain/AD")